# Mushroom Stew

## Develop a mushroom stew, that is visually appealing, pleasingly smelling, and preferably non-toxic.

* Explore the fields, which ones could affect the taste or smell, which can be ignored? 
* Which fields may affect if the stew is visually appealing? 
* Use graphics to support your choices

## Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import plotly.express as px
#import geopandas
from pandas_profiling import ProfileReport
from urllib.request import urlopen
import json
import plotly.graph_objects as go
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tools.eval_measures import rmse
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, RidgeCV, Lasso
import sklearn.metrics as metrics
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor, plot_importance

## Load Dataset, Explore and Display Features

In [ ]:
col_names=['class','cap-shape', 'cap-surface', 'cap-color', 'bruises?', 'odor','gill-attachment',\
           'gill-spacing','gill-size','gill-color','stalk-shape','stalk-root','stalk-surface-above-ring',\
           'stalk-surface-below-ring','stalk-color-above-ring','stalk-color-below-ring','veil-type',\
           'veil-color','ring-number','ring-type','spore-print-color','population','habitat'] 
mushroom_df = pd.read_csv('agaricus-lepiota.csv', names=col_names, header=None)

In [ ]:
pd.set_option("display.max_columns", None)
mushroom_df.head(5)

In [ ]:
mushroom_df.dtypes

In [ ]:
mushroom_df.describe()

In [ ]:
profile = ProfileReport(mushroom_df)
profile